In [ ]:
%matplotlib inline

# Pandas
<!-- requirement: img/Data_Frame_Data_Series.png -->
<!-- requirement: small_data/fha_by_tract.csv -->
<!-- requirement: small_data/2013_Gaz_tracts_national.tsv -->
Pandas es la solución que Python propone frente a R. Es una buena tecnología para tratar conjuntos de datos "pequeños", 
es decir, aquellos que caben en la memória RAM del ordenador.

El concepto básico que introduce pandas es el de **data frame**.

## Nouns (objects) en Pandas

### Data Frames

Como una tabla, con filas y columnas (por ejemplo, como en SQL). 
Excepto:
   - Las filas pueden ser indexadas por algo interesante (hay soporte especial para etiquetas como datos categóricos y de series temporales). Esto es especialmente útil cuando tiene datos de series temporales con puntos de datos potencialmente faltantes.
   - Las celdas pueden almacenar objetos de Python. Al igual que en un excel, las columnas son homogéneas.
   - En lugar de "NULL", el nombre para un valor inexistente es "NA". A diferencia de R, los dataframes de Python solo admiten NA en columnas de algunos tipos de datos (básicamente: números en coma flotante y 'objetos'), pero esto no es un problema en su mayor parte.
  
### Serie de datos:

Estas son columnas nombradas de un DataFrame (más correctamente, un dataframe es un diccionario de Series). Las entradas de la serie tienen un tipo homogéneo.

<img src="img/4-pandas/Data_Frame_Data_Series.png">

In [1]:
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

#import re # expresiones regulares

# un data frame
df1 = pd.DataFrame({
    'number': [1, 2, 3],
    'animal': ['cat', 'dog', 'mouse']
})

df1

,animal,number
0,cat,1
1,dog,2
2,mouse,3


In [2]:
# seleccionamos la columna animal
df1['animal']

0      cat
1      dog
2    mouse
Name: animal, dtype: object

In [ ]:
df1.animal

In [3]:
# mostramos los tipos de datos de las columnas
df1.dtypes

animal    object
number     int64
dtype: object

In [5]:
df1['number'] = df1['number'].astype(float)

In [6]:
df1

,animal,number
0,cat,1.0
1,dog,2.0
2,mouse,3.0


In [4]:
# Creamos un dataframe igual
df2 = pd.DataFrame([
    ('cat', 1),
    ('dog', 3),
    ('mouse', 3),
], columns=['animal', 'number'])

np.all(df1 == df2)

False

## Verbos (operaciones) en Pandas
  
Pandas proporciona un análisis básico de datos "con pilas incluidas":
  - ** Cargando datos: ** `read_csv`,` read_table`, `read_sql`, y ` read_html`
  - ** Selección, filtrado y agregación ** (es decir, operaciones de tipo SQL): hay una sintaxis especial para seleccionar. Existe el método `merge`. También hay una sintaxis fácil para crear una columna nueva cuyo valor se calcula desde otra columna, con la ventaja que los cálculos pueden usar toda la potencia de Python (aunque podría ser más rápido si no lo hiciera ;) ).
  - ** Agregación de estilo de "tabla pivote": ** Si eres un experto en Excel, puedes apreciar esto.
  - ** NA handling: ** Al igual que los dataframes de R, hay un buen soporte para transformar los valores de NA con valores por defecto / trucos de promedios / etc.
  - ** Estadísticas básicas: ** `mean`, ` median`, `max`,` min`, y el siempre útil `describe`.
  - ** Conectando a análisis más avanzados: ** Esto no se incluye por defecto. Pero aún así, se entiende razonablemente bien con `sklearn`.
  - ** Visualización: ** Por ejemplo `plot` y `hist`. Tambien podemos usar matplotlib y también seaborn.
  
Examinaremos un poco sobre todos estos en el contexto de un ejemplo.

Vamos a explorar un conjunto de datos de seguro hipotecario emitido por la Autoridad Federal de Vivienda (FHA). Los datos se desglosan por tramo censal y nos dice qué tamaño tiene la FHA en cada tramo (cuántas casas, etc.).

## Carga de datos y primera toma de contacto

**Podemos leer un csv y decirle que nombre tiene cada columna**

In [18]:
names =["State_Code", "County_Code", "Census_Tract_Number",
        "NUM_ALL", "NUM_FHA", "PCT_NUM_FHA", "AMT_ALL",
        "AMT_FHA", "PCT_AMT_FHA"]

df = pd.read_csv('fha_by_tract.csv', names=names)  # Cargamos un archivo csv

df.head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA
0,8.0,75.0,NaN,1,1,100.0,258,258,100.0
1,28.0,49.0,103.01,1,1,100.0,71,71,100.0
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0
3,39.0,113.0,603.00,3,3,100.0,206,206,100.0
4,12.0,105.0,124.04,2,2,100.0,303,303,100.0


**O también asignar los nombres a las columnas a posteriori**

In [21]:

names =["StateCode", "County_Code", "Census_Tract_Number",
        "NUM_ALL", "NUM_FHA", "PCT_NUM_FHA", "AMT_ALL",
        "AMT_FHA", "PCT_AMT_FHA"]

df.columns = names


In [23]:

df = df.rename(columns={"StateCode":"State_Code"}) # incluso lo podemos hacer una columna en particular

In [ ]:
df.head() #Probad de pasar un número por parámetro

**Crear una nueva columna como combinación de otras:** 'Census_Tract_Number', 'County_Code' y 'State_Code'


In [26]:
df['GEOID'] = df['Census_Tract_Number']*100 + 10**6 * df['County_Code'] + 10**9 * df['State_Code']   
df.head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
0,8.0,75.0,NaN,1,1,100.0,258,258,100.0,NaN
1,28.0,49.0,103.01,1,1,100.0,71,71,100.0,2.804901e+10
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0,NaN
3,39.0,113.0,603.00,3,3,100.0,206,206,100.0,3.911306e+10
4,12.0,105.0,124.04,2,2,100.0,303,303,100.0,1.210501e+10


Si queremos eliminar una columna:

In [28]:
column_to_drop = 'GEOID'
df.drop(column_to_drop, axis = 1).head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA
0,8.0,75.0,NaN,1,1,100.0,258,258,100.0
1,28.0,49.0,103.01,1,1,100.0,71,71,100.0
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0
3,39.0,113.0,603.00,3,3,100.0,206,206,100.0
4,12.0,105.0,124.04,2,2,100.0,303,303,100.0


La mayoría de las operaciones producen copias (a menos que se especifique `inplace = True`). El objeto `df` todavía tiene la columna GEOID.

In [30]:
# miramos si la columna aún esta en el dataframe
column_to_drop in df.columns

True

In [ ]:
# Usar inplace=True no es recomendable, es mejor asignar el nuevo dataframe a una nueva variable.

df_new = df.drop(column_to_drop, axis = 1)

print(column_to_drop in df.columns)

print(column_to_drop in df_new.columns)

Las filas también se pueden eliminar. Los índices no se reinician. El índice está asociado con la fila, no con el orden.

In [ ]:
df.drop(0, axis=0).head()

Por defecto, las filas están indexadas por su posición. Sin embargo, cualquier columna se puede convertir en un índice:

In [68]:
df.set_index('State_Code').head()

,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
State_Code,,,,,,,,,
8.0,75.0,NaN,1,1,100.0,258,258,100.0,NaN
28.0,49.0,103.01,1,1,100.0,71,71,100.0,2.804901e+10
40.0,3.0,NaN,1,1,100.0,215,215,100.0,NaN
39.0,113.0,603.00,3,3,100.0,206,206,100.0,3.911306e+10
12.0,105.0,124.04,2,2,100.0,303,303,100.0,1.210501e+10


Podemos indexar a multiples niveles:

In [78]:
df.set_index(['State_Code', 'County_Code']).head(10)

Census_Tract_Number  NUM_ALL  NUM_FHA  PCT_NUM_FHA  \
State_Code County_Code                                                       
8.0        75.0                         NaN        1        1        100.0   
28.0       49.0                      103.01        1        1        100.0   
40.0       3.0                          NaN        1        1        100.0   
39.0       113.0                     603.00        3        3        100.0   
12.0       105.0                     124.04        2        2        100.0   
           86.0                     9808.00        1        1        100.0   
39.0       35.0                     1202.00        1        1        100.0   
12.0       103.0                     207.00        2        2        100.0   
36.0       119.0                      30.00        1        1        100.0   
39.0       153.0                        NaN        1        1        100.0   

                        AMT_ALL  AMT_FHA  PCT_AMT_FHA         GEOID  
State_Code County_Code                                               
8.0        75.0             258      258        100.0           NaN  
28.0       49.0              71       71        100.0  2.804901e+10  
40.0       3.0              215      215        100.0           NaN  
39.0       113.0            206      206        100.0  3.911306e+10  
12.0       105.0            303      303        100.0  1.210501e+10  
           86.0             188      188        100.0  1.208698e+10  
39.0       35.0              19       19        100.0  3.903512e+10  
12.0       103.0            100      100        100.0  1.210302e+10  
36.0       119.0            354      354        100.0  3.611900e+10  
39.0       153.0            213      213        100.0           NaN

Y podemos volver atrás:

In [ ]:
df.set_index('State_Code').reset_index().head()

In [33]:
# Podemos describir una columna
print("Percentage of mortages in each census tract insured by FHA")
df['PCT_AMT_FHA'].describe()

Percentage of mortages in each census tract insured by FHA


count    72035.000000
mean        29.703179
std         24.037779
min          0.000000
25%         10.780800
50%         24.753900
75%         44.207550
max        100.000000
Name: PCT_AMT_FHA, dtype: float64

In [34]:
# O el dataframe entero
df.describe()

/home/biel/anaconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
count,72034.000000,71984.000000,71040.000000,72035.000000,72035.000000,72035.000000,7.203500e+04,72035.000000,72035.000000,7.104000e+04
mean,28.322528,85.612636,2534.598023,36.970389,9.741667,28.566878,7.886092e+03,1689.278851,29.703179,2.837319e+10
std,16.459507,98.672445,3451.173223,53.975403,15.187832,22.404545,1.302542e+04,2800.346300,24.037779,1.648784e+10
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,2.000000e+00,0.000000,0.000000,1.001020e+09
25%,NaN,NaN,NaN,13.000000,2.000000,11.111100,1.551000e+03,281.000000,10.780800,NaN
50%,NaN,NaN,NaN,27.000000,6.000000,25.000000,4.168000e+03,932.000000,24.753900,NaN
75%,NaN,NaN,NaN,48.000000,13.000000,41.935500,9.668000e+03,2197.000000,44.207550,NaN
max,72.000000,840.000000,9922.010000,9477.000000,1932.000000,100.000000,1.575871e+06,331515.000000,100.000000,7.215375e+10


In [ ]:
%matplotlib inline

In [ ]:
# Dibujar el histograma de una columna.
df['PCT_AMT_FHA'].plot(kind='hist', bins=100); #Probad a poner y quitar el ;

La distribución anterior parece sesgada, así que veamos su logaritmo.

In [ ]:
df['LOG_AMT_ALL'] = np.log1p(df['AMT_ALL'])  # Creamos una nueva columna
print(df['LOG_AMT_ALL'].describe())

df['AMT_ALL'].apply(np.log1p).hist(bins=500);  # Or apply a function to each element

## Indexando un dataframe

Indexar por un nombre de columna produce una serie de datos.

In [51]:
df['State_Code'].head()

0     8.0
1    28.0
2    40.0
3    39.0
4    12.0
Name: State_Code, dtype: float64

Indexar por una lista de nombres de columna da otro dataframe.

In [53]:
df[['State_Code', 'County_Code']].head()

,State_Code,County_Code
0,8.0,75.0
1,28.0,49.0
2,40.0,3.0
3,39.0,113.0
4,12.0,105.0


**Pregunta:** Que nos devolverá?

In [54]:
type(df[['State_Code']])

pandas.core.frame.DataFrame

In [36]:
df[['State_Code']].head()

,State_Code
0,8.0
1,28.0
2,40.0
3,39.0
4,12.0


Un dataframe es un iterador que nos devuelve el nombre de las columnas

In [77]:
for col in df:
    print(col)

# De forma avanzada, generamos una lista de columnas
[col for col in df]

State_Code
County_Code
Census_Tract_Number
NUM_ALL
NUM_FHA
PCT_NUM_FHA
AMT_ALL
AMT_FHA
PCT_AMT_FHA
GEOID


['State_Code',
 'County_Code',
 'Census_Tract_Number',
 'NUM_ALL',
 'NUM_FHA',
 'PCT_NUM_FHA',
 'AMT_ALL',
 'AMT_FHA',
 'PCT_AMT_FHA',
 'GEOID']

Hasta ahora hemos visto como seleccionar columnas, pero no hemos visto como seleccionar filas de nuestro conjunto de datos

In [37]:
df[:3]

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
0,8.0,75.0,NaN,1,1,100.0,258,258,100.0,NaN
1,28.0,49.0,103.01,1,1,100.0,71,71,100.0,2.804901e+10
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0,NaN


Para indexar un elemento particular del dataframe, usaremos el atributo `.loc`.  Que toma como parámetroíndice y columna.

In [80]:
auxf = df.set_index('State_Code')
auxf.loc[28.0,:].head(40)

,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
State_Code,,,,,,,,,
28.0,49.0,103.01,1,1,100.0000,71,71,100.0000,2.804901e+10
28.0,77.0,NaN,1,1,100.0000,177,177,100.0000,NaN
28.0,63.0,9502.00,1,1,100.0000,133,133,100.0000,2.806395e+10
28.0,49.0,25.00,1,1,100.0000,82,82,100.0000,2.804900e+10
28.0,151.0,12.00,3,3,100.0000,118,118,100.0000,2.815100e+10
28.0,151.0,21.00,2,2,100.0000,107,107,100.0000,2.815100e+10
28.0,49.0,22.00,1,1,100.0000,39,39,100.0000,2.804900e+10
28.0,103.0,9502.00,1,1,100.0000,165,165,100.0000,2.810395e+10
28.0,49.0,27.00,1,1,100.0000,78,78,100.0000,2.804900e+10


In [81]:
df.loc[3, 'State_Code']

39.0

Inusualmente para Python, ambos puntos finales se incluyen en el sector.

In [ ]:
df.loc[0:3, ['State_Code','Census_Tract_Number']]

La indexación basada en la posición se realiza con el atributo `.iloc`.

In [ ]:
df.iloc[3, 0:3]

La convención de corte habitual se utiliza para `.iloc`. Es decir, el extremo superior no se incluye en el segmento

In [83]:
df.iloc[0:3, 0:3]

,State_Code,County_Code,Census_Tract_Number
0,8.0,75.0,NaN
1,28.0,49.0,103.01
2,40.0,3.0,NaN


## Filtrado de información

 La notación `df [...]` es muy flexible:
   - Acepta nombres de columnas (cadenas y listas de cadenas);
   - Acepta los números de las columnas (siempre que no haya ambigüedad con los nombres de las columnas);
   - ¡Acepta series de datos binarias! 
  
Esto significa que puedes escribir:
```python

 df[ df['column_name2'] == 'MD' & ( df['column_name1']==5 | df['column_name1']==6 ) ]
```   
para los que sepan SQL seria una cosa muy similar a:
```sql
SELECT * FROM df
WHERE column_name2="MD" AND (column_name1=5 OR column_name1=6)
```           
Los operadores booleanos en un dataframe devuelven una serie de datos de bools.

In [84]:
(df['State_Code'] == 1).head()

0    False
1    False
2    False
3    False
4    False
Name: State_Code, dtype: bool

Estos se pueden combinar con los operadores booleanos (a nivel de bit). Tened en cuenta que, debido a la precedencia del operador, es mejor poner las comparaciones individuales entre paréntesis.

In [ ]:
((df['State_Code'] == 1) & (df['Census_Tract_Number'] == 9613)).head()

Los dataframes pueden ser indexados por series de booleanos

In [ ]:
df[df['State_Code'] == 5][['State_Code', 'County_Code']].head()

** Nota: ** selecciona filas por series de datos binarios solo si comparten el mismo índice de datos.

### **Ejercicio:**
1. 

## Uniendo datos (joining)

Lo análogo a
>             
    SELECT * 
        FROM df1
        INNER JOIN df2 
        ON df1.field_name=df2.field_name;

es

    df_joined = df1.merge(df2, on='field_name')

También  se pueden hacer joins izquierda / derecha / outter, mezclar y combinar nombres de columna, etc. [Documentación Pandas](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)

Por supuesto, solo mirando la distribución del seguro por zona censal no es interesante a menos que sepamos más sobre el tramo censal.

In [ ]:
df.head()

In [85]:
df_geo = pd.read_csv('2013_Gaz_tracts_national.tsv', sep='\t')
df_geo.head()

,USPS,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,1001020100,9809939,36312,3.788,0.014,32.481794,-86.490249
1,AL,1001020200,3340498,5846,1.290,0.002,32.475758,-86.472468
2,AL,1001020300,5349274,9054,2.065,0.003,32.474024,-86.459703
3,AL,1001020400,6382705,16244,2.464,0.006,32.471030,-86.444835
4,AL,1001020500,11397734,48412,4.401,0.019,32.458916,-86.421817


In [86]:
df_joined = df.merge(df_geo, on='GEOID', how='left')
df_joined.head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID,USPS,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,8.0,75.0,NaN,1,1,100.0,258,258,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28.0,49.0,103.01,1,1,100.0,71,71,100.0,2.804901e+10,MS,8162270.0,22648.0,3.151,0.009,32.365904,-90.262379
2,40.0,3.0,NaN,1,1,100.0,215,215,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39.0,113.0,603.00,3,3,100.0,206,206,100.0,3.911306e+10,OH,5382347.0,0.0,2.078,0.000,39.729932,-84.268323
4,12.0,105.0,124.04,2,2,100.0,303,303,100.0,1.210501e+10,FL,105120002.0,1800522.0,40.587,0.695,28.224489,-81.739745


## Agregando información

El análogo al SQL `GROUP BY` es

    grouped = df.groupby(['field_name1', ...])...

The above is analogous to
>             
    SELECT mean(df.value1), std(df.value2) 
        FROM df
        GROUP BY df.field_name1, ...

Pandas es algo más flexible en cuanto a cómo agrupar los datos.

In [87]:
usps_groups = df_joined.groupby('USPS')
usps_groups

La razón por la que Pandas no requiere que especifiques una función de agregación por adelantado es porque el método groupby por sí solo ya realiza parte del trabajo. Devuelve un tipo de datos `DataFrameGroupBy` que contiene un diccionario de claves de grupo para las listas.

In [88]:
print(type(usps_groups.groups))
usps_groups.groups['AK'][:5] # Vemos los 5 primeros

<class 'dict'>


[91, 298, 536, 2528, 4146]

In [89]:
usps_groups.groups.keys()

dict_keys(['LA', 'NJ', 'TX', 'SD', 'DE', 'NC', 'NY', 'UT', 'WV', 'CT', 'PA', 'MN', 'MI', 'OK', 'WY', 'KS', 'VT', 'AL', 'IN', 'AZ', 'WI', 'IA', 'SC', 'IL', 'RI', 'MO', 'FL', 'ID', 'OH', 'KY', 'DC', 'NH', 'AR', 'MD', 'MT', 'NM', 'OR', 'VA', 'MS', 'ND', 'NV', 'ME', 'NE', 'WA', 'AK', 'PR', 'MA', 'TN', 'GA', 'CO', 'CA', 'HI'])

Podemos recuperar el grupo de datos asociados con una clave:

In [90]:
usps_groups.get_group('AK')[:5]

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID,USPS,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
91,2.0,50.0,3.0,1,1,100.0000,121,121,100.0000,2.050000e+09,AK,4.930719e+10,6.258804e+08,19037.615,241.654,61.468804,-156.692206
298,2.0,20.0,4.0,1,1,100.0000,192,192,100.0000,2.020000e+09,AK,5.336808e+07,2.393787e+07,20.606,9.242,61.266668,-149.830858
536,2.0,122.0,1.0,1,1,100.0000,39,39,100.0000,2.122000e+09,AK,1.982658e+10,1.006239e+10,7655.087,3885.112,60.123779,-152.841141
2528,2.0,185.0,1.0,11,8,72.7273,1958,1604,81.9203,2.185000e+09,AK,2.532416e+08,3.781985e+08,97.777,146.023,71.287938,-156.685618
4146,2.0,122.0,12.0,3,2,66.6667,530,392,73.9623,2.122001e+09,AK,2.498757e+09,4.721135e+09,964.775,1822.840,59.356474,-151.771450


Esto es lo mismo que:

In [ ]:
df_joined.iloc[usps_groups.groups['AK'][:5]]

In [ ]:
usps_groups.mean().head()  #Proporciona la media de cada grupo

In [92]:
df_by_state = df_joined.groupby('USPS').sum()

In [ ]:
df_by_state

In [ ]:
# This is the analog of
# SELECT USPS, SUM(AMT_FHA), SUM(AMT_ALL), ... FROM df GROUP BY USPS;
df_by_state = usps_groups['AMT_FHA', 'AMT_ALL', 'NUM_FHA', 'NUM_ALL'].sum()
df_by_state.head()

In [ ]:
df_by_state['PCT_AMT_FHA'] = 100.0 * df_by_state['AMT_FHA']  / df_by_state['AMT_ALL']

# Se puede observar la diferencia respecto al histograma de tracto censal
df_by_state['PCT_AMT_FHA'].hist(bins=20);

También puedes especificar una función de agregación para cada columna:

In [ ]:
usps_groups['NUM_FHA', 'NUM_ALL'].agg({'NUM_FHA': np.sum, 'NUM_ALL': np.mean}).head()

La función **groupby** es especialmente útil cuando definimos nuestras propias funciones de agregación. Aquí, definimos una función que devuelve la fila para la pista del censo ubicada más al norte. La función de aplicar intenta 'combinar resultados de una manera inteligente'. La lista de objetos Serie de cada llamada a `farthest_north` para cada código USPS se contrae en una sola tabla DataFrame.

In [ ]:
def farthest_north(state_df):
    # descending sort, then select row 0
    # the datatype will be a pandas Series
    return state_df.sort_values('INTPTLAT', ascending=False).iloc[0]

df_joined.groupby('USPS').apply(farthest_north)[:10]

## Ordenando por índices y columnas

Podemos ordenar por el índice


In [93]:
df_by_state.sort_index(ascending=False).head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
USPS,,,,,,,,,,,,,,,,
WY,7224.0,2793.0,674510.46,6644,1418,2547.593870,1313697,248573,2603.529903,7.226860e+12,2.514528e+11,1.862544e+09,97086.470,719.139,5504.476940,-13801.565936
WV,26136.0,25338.0,1778944.91,11739,2129,9271.010760,1675079,282703,9584.306890,2.616152e+13,6.226658e+10,4.894455e+08,24041.264,188.978,18780.862461,-39110.042522
WI,75295.0,104154.0,3317836.54,47913,7104,22031.121552,7500688,971178,22901.272090,7.539949e+13,1.401892e+11,8.509759e+09,54127.361,3285.636,59885.350451,-121833.117630
WA,76055.0,60321.0,2525773.28,68902,15019,33769.050593,17267705,3201463,34514.086295,7.611557e+13,1.711495e+11,9.474026e+09,66081.203,3657.953,67911.362121,-174531.160542
VT,9100.0,2728.0,1309303.31,4777,421,1738.821260,940893,77355,1744.257130,9.102859e+12,2.386749e+10,1.034339e+09,9215.295,399.366,8013.192295,-13244.209002


También podemos ordenar según el valor de una columna

In [102]:
df_by_state.sort_values('AMT_FHA', ascending=False).head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
USPS,,,,,,,,,,,,,,,,
CA,47058.0,429621.0,1.572751e+07,307087,88650,231906.640878,99817500,22607179,258551.356138,4.748919e+13,3.845024e+11,6.572385e+09,148457.258,2537.612,278462.046524,-936564.108106
TX,241584.0,1147425.0,1.328567e+07,243977,73549,175000.372320,45870174,10789870,179065.734095,2.427328e+14,6.518961e+11,1.416095e+10,251698.524,5467.578,156128.381067,-490564.133622
FL,48864.0,292224.0,2.845053e+06,157431,44511,131126.352219,30754332,6866263,134137.105306,4.915651e+13,1.314206e+11,1.311422e+10,50741.788,5063.422,113139.351341,-332655.086780
NY,165780.0,288479.0,5.743982e+06,101335,25019,135397.247540,30631391,5578590,143698.707311,1.660691e+14,1.204434e+11,5.656721e+09,46503.460,2184.083,191221.672549,-343856.056861
VA,94146.0,543493.0,4.895165e+06,85169,20495,45519.182440,23048658,4713396,46582.061165,9.468998e+13,1.018544e+11,4.499377e+09,39326.196,1737.214,69750.877112,-143608.120106


## Valores únicos


En Pandas podemos contar el número de valores únicos, repeticiones i testear la pertenencia:

In [ ]:
df['State_Code'].unique()[:10]

In [ ]:
df['State_Code'].value_counts().head()

In [ ]:
df[df['State_Code'].isin(df['State_Code'].head(3))] #Seleccionamos según la pertenencia

## Tratamiento de información faltante y NA

Cuando leemous en  archivo CSV o una base de datos SQL, a menudo encontramos valores "NA" (o "nulo", "Ninguno", etc.). El lector CSV tiene un campo especial para especificar cómo se denota esto, y SQL tiene la noción incorporada de NULL. Pandas proporciona algunas herramientas para trabajar con estos; generalmente son similares a (y un poco peor que) `R`.

Debemos tener en cuenta que estos métodos no lo hacen ´inplace´, es decir, crean una nueva serie y no cambian la original.
For more details: http://pandas.pydata.org/pandas-docs/stable/missing_data.html

In [ ]:
df['GEOID'][:10]

`.isnull()` y `.notnull()` examinan si existe algún null y devuelven una lista.

In [ ]:
df['GEOID'].isnull()[:10]

`.dropna()`  elimina las filas con información nula.

In [ ]:
df['GEOID'].size, df['GEOID'].dropna().size

`.fillna()` substituye los valores N/A con otro valor.  `.interpolate()` substituye los valores nulos con una interpolación (linear, or quadratic, or...). 

In [ ]:
df['fill_0'] = df['GEOID'].fillna(0)                          # Fills constant value, here 0
df['fill_forward'] = df['GEOID'].fillna(method='ffill')       # Fill forwards
df['fill_back'] = df['GEOID'].fillna(method='bfill', limit=5) # Fill backwards, at most 5
df['fill_mean'] = df['GEOID'].fillna(df['GEOID'].mean())      # Fills constant value, here the mean (imputation)
df['fill_interp'] = df['GEOID'].interpolate()                 # Fills interpolated value
df[['GEOID', 'fill_0', 'fill_forward', 'fill_back', 'fill_mean', 'fill_interp']][:10]

### Nota
Los valores N / A (generalmente) se ignoran inteligentemente al realizar otros cálculos en dataframes. Por ejemplo, cuando se usan métodos de cadena en series:

In [ ]:
text_series = df['GEOID'].replace(0, np.nan).apply(str)
print(text_series[:10])

In [ ]:
text_series[:10].str.split('.')

La aplicación de la media en los datos numéricos ignora los NA por defecto (consultar la documentación):

In [98]:
df['GEOID'].mean()

28373187313.969

## Manipulación de strings

Las operaciones de string a nivel de elemento están disponibles a través del atributo `.str`.

[más información](https://pandas.pydata.org/pandas-docs/stable/text.html)

In [97]:
states = df_joined['USPS'].dropna()
states[states.str.contains('A')].head()


12    GA
20    GA
23    LA
26    AZ
27    GA
Name: USPS, dtype: object

## Indices en Pandas

Los índices de pandas nos permiten manejar los datos de forma natural. ** Como hemos comentado antes, los elementos se asocian en función de su índice, no de su orden. **

In [100]:
s1 = pd.Series([1,2,3], index=['a', 'b', 'c'])
s2 = pd.Series([3,2,1], index=['c', 'b', 'a'])
s1 + s2

a    2
b    4
c    6
dtype: int64

In [101]:
s3 = pd.Series([3,2,1], index=['c', 'd', 'e'])
s1 + s3

a    NaN
b    NaN
c    6.0
d    NaN
e    NaN
dtype: float64

Los valores faltantes obtienen un NaN, pero esto puede ser reemplazado por un valor de relleno de nuestra elección.

In [ ]:
s1.add(s3, fill_value=0)

## Function application and mapping

Para la aplicación de funciones a nivel de elemento, lo más sencillo es aplicar funciones **numpy** a estos objetos:

In [ ]:
df1 = pd.DataFrame(np.arange(24).reshape(4,6))

np.sin(df1)

Esto se basa en funciones numpy que se transmiten automáticamente para trabajar en función de los elementos. Para aplicar una función Python a cada elemento, use el método `.applymap ()`.

In [ ]:
df1.applymap(lambda x: "%.2f" % x)

Sin embargo, a veces desea calcular cosas en columnas o filas. En este caso, deberá usar el método `apply`.

Por ejemplo, el siguiente código muestra el rango de valores de cada columna.

In [ ]:
df1.apply(lambda x: x.max() - x.min())

## Realizando análisis más avanzados

Casi cualquier herramienta de "análisis avanzado" en el ecosistema de Python va a tomar matrices de tipo `np.array` como entrada. Puede acceder a la matriz subyacente de una columna de un data frame como

         df ['column']. values
        
Muchos de ellos toman `nd.array` a cuyos datos subyacentes se puede acceder mediante

         df.values
        
directamente. * La mayoría * de las veces, tomarán `df ['column']` y `df` sin necesidad de mirar los valores.

Esto es particularmente importante si desea usar Pandas con la biblioteca sklearn. Consultad esta [publicación](http://www.markhneedham.com/blog/2013/11/09/python-making-scikit-learn-and-pandas-play-nice/) para ver un ejemplo.

In [ ]:
df1.apply(lambda x: x.max() - x.min(), axis=1)

# Extra !!!

## Pandas Timestamps

Pandas viene con excelentes herramientas para administrar datos temporales. El elemento central de esto es la clase Timestamp, que puede inferir marcas de tiempo de muchas entradas diferentes:

In [ ]:
print(pd.Timestamp('July 4, 2016'))
print(pd.Timestamp('Monday, July 4, 2016'))
print(pd.Timestamp('Tuesday, July 4th, 2016'))  # notice it ignored 'Tuesday'
print(pd.Timestamp('Monday, July 4th, 2016 05:00 PM'))
print(pd.Timestamp('04/07/2016T17:20:13.123456'))
print(pd.Timestamp(1467651600000000000))  # number of ns since the epoch, 1/1/1970

Tambien con zonas horarias:

In [ ]:
july4 = pd.Timestamp('Monday, July 4th, 2016 05:00 PM').tz_localize('US/Eastern')
labor_day = pd.Timestamp('9/5/2016 12:00', tz='US/Eastern')
thanksgiving = pd.Timestamp('11/24/2016 16:00')  # no timezone

Pandas puede hacer cálculos en Timestamps si están localizados en la misma zona horaria o ninguno tiene una zona horaria.

In [ ]:
print(labor_day - july4)
# con  thanksgiving - july4  # obtendriamos un error

Los desplazamientos de series de tiempo son útiles para calcular fechas relativas a otra fecha. Observad que omite durante los días de fin de semana, pero es ajeno a las vacaciones. Pandas admite [calendarios personalizados] (http://pandas.pydata.org/pandas-docs/stable/timeseries.html#holidays-holiday-calendars) si los necesitamos.

In [ ]:
from pandas.tseries.offsets import BDay, Day, BMonthEnd

print(july4 + Day(5))  # 5 calendar days later, a Saturday.
print(july4 + BDay(5))  # 5 business days later, or the following Monday.
print(july4 - BDay(1))  # 1 business day earlier, or the previous Friday.
print(july4 + BMonthEnd(1))  # last business day of the month.

Pandas puede generar rangos de fechas. Aquí generamos una lista de los días de trabajo de enero de 2016:

In [ ]:
business_days = pd.date_range('1/1/2016', '1/31/2016', freq='B')
business_days

Esto a su vez puede usarse como un índice de DataFrame:

In [ ]:
time_df = pd.DataFrame(np.random.rand(len(business_days)),
                    index=business_days,
                   columns=['random'])
time_df.head()

Las funciones de zona horaria siguen siendo usables:

In [ ]:
time_df.tz_localize('UTC').tz_convert('US/Pacific').head()

## Multi-indices, stacking, and pivot tables

Data frames pueden contener múltiples índices para filas o columnas. Por ejemplo, la agrupación por dos columnas producirá un índice de fila de dos niveles.

In [ ]:
grouped = df.groupby(['State_Code', 'County_Code'])[['NUM_ALL', 'NUM_FHA']].sum()
grouped.head()

Un índice de fila pues ser transformado a un índice de columna con el método `.unstack()`:

In [ ]:
grouped.unstack().head()

El método `.stack()` realiza el trabajo contrario:

In [ ]:
np.all(grouped.unstack().stack() == grouped)

Esto puede llevarse a cabo con la función `pivot_table()` function.

In [ ]:
pd.pivot_table(df, index='State_Code', columns='County_Code',
               values=['NUM_ALL', 'NUM_FHA'], aggfunc=np.sum).head()

You may already by familiar with pivot tables in Excel.  These work similarly, and area  good tool for changing the dependent and independent variables for aggregations of data. See http://pandas.pydata.org/pandas-docs/stable/reshaping.html for more information.

### Pandas HTML data import example

Pandas takes a "batteries included" approach and throws in a whole lot of convenience functions.  For instance it has import functions for a variety of formats.  One of the pleasant surprises is a command `read_html` that's meant to automate the process of extracting tabular data from HTML.  In particular, it works pretty well with tables on Wikipedia.  

Let's do an example: We'll try to extract the list of the world's tallest structures from
http://en.wikipedia.org/wiki/List_of_tallest_buildings_and_structures_in_the_world.

In [94]:
dfs = pd.read_html('http://en.wikipedia.org/wiki/List_of_tallest_buildings_and_structures_in_the_world', header=0, parse_dates=False)

# There are several tables on the page.  By inspection we can figure out which one we want
tallest = dfs[3]
print(tallest.columns)
# The coordinates column needs to be fixed up.  This is a bit of string parsing:
def clean_lat_long(s):
    try:
        parts = s.split("/")
    except AttributeError:
        return (None, None)
    if len(parts) < 3:
        return (None, None)
    m = re.search(r"(\d+[.]\d+);[^\d]*(\d+[.]\d+)[^\d]", parts[2])
    if not m:
        return (None, None)
    return (m.group(1), m.group(2))

tallest['Clean_Coordinates'] = tallest['Coordinates'].apply(clean_lat_long)
tallest['Latitude'] = tallest['Clean_Coordinates'].apply(lambda x:x[0])
tallest['Longitude'] = tallest['Clean_Coordinates'].apply(lambda x:x[1])

# Et voila
tallest.head()

Index(['Category', 'Structure', 'Country', 'City', 'Height (metres)',
       'Height (feet)', 'Year built', 'Coordinates'],
      dtype='object')


,Category,Structure,Country,City,Height (metres),Height (feet),Year built,Coordinates,Clean_Coordinates,Latitude,Longitude
0,Mixed-use,Burj Khalifa,United Arab Emirates,Dubai,829.8,2722.0,2010,25°11′50.0″N 55°16′26.6″E﻿ / ﻿25.197222°N 55.2...,"(25.197222, 55.274056)",25.197222,55.274056
1,Self-supporting tower,Tokyo Skytree,Japan,Tokyo,634,2080.0,2011,35°42′36.5″N 139°48′39″E﻿ / ﻿35.710139°N 139.8...,"(35.710139, 139.81083)",35.710139,139.81083
2,Guyed steel lattice mast,KVLY-TV mast,United States,"Blanchard, North Dakota",628.8,2063.0,1963,47°20′32″N 97°17′25″W﻿ / ﻿47.34222°N 97.29028°...,"(47.34222, 97.29028)",47.34222,97.29028
3,Office,One World Trade Center,United States,"New York, New York",541,1776.0,2013,40°42′46.8″N 74°0′48.6″W﻿ / ﻿40.713000°N 74.01...,"(40.713000, 74.013500)",40.713000,74.013500
4,Military structure,Large masts of INS Kattabomman,India,"Tirunelveli, Tamil Nadu",471,1545.0,2014,8°22′42.52″N 77°44′38.45″E﻿ / ﻿8.3784778°N 77....,"(8.3784778, 77.7440139)",8.3784778,77.7440139


**Exercise**

1. Parse the table rankings of [UK universities available on Wikipedia](https://en.wikipedia.org/wiki/Rankings_of_universities_in_the_United_Kingdom):